In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
import pickle

# Load dataset
data = pd.read_csv('/content/gynecological_conditions.csv')

# View dataset columns to understand the structure
print(data.columns)




Index(['Disorder', 'Symptoms', 'Treatment Recommendation',
       'Food Intake Recommendation', 'Precautions', 'Foods to Avoid',
       'Duration of Symptoms', 'Lifestyle Recommendations'],
      dtype='object')


In [2]:
!pip install PyMuPDF


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 76.9 MB/s eta 0:00:00


In [4]:
#uploading pdf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import fitz  # PyMuPDF for PDF text extraction

# Load dataset
data = pd.read_csv('/content/gynecological_conditions.csv')

# View dataset columns to understand the structure
print(data.columns)

# Define input features and targets
X = data['Symptoms']  # Input features (Symptoms)

# Targets for prediction
targets = ['Disorder', 'Treatment Recommendation', 'Precautions', 'Food Intake Recommendation']
target_data = data[targets]

# Encoding targets
le_targets = {}
for target in targets:
    le_targets[target] = LabelEncoder()
    target_data[target] = le_targets[target].fit_transform(target_data[target])

# Split the dataset into training and testing sets for the features
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Get the corresponding target values for the training and testing sets
y_train = target_data.loc[X_train.index]
y_test = target_data.loc[X_test.index]

# Vectorize the symptoms data
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train separate Naive Bayes models for each target
models = {}
for target in targets:
    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train[target])  # Using y_train[target] to match the length of X_train_vectorized
    models[target] = model

# Save the trained models and vectorizer to files
with open('naive_bayes_treatment_model.pkl', 'wb') as model_file:
    pickle.dump(models, model_file)

with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

# Function to predict disorder, treatment, precautions, and food intake based on symptoms input
def predict_outputs(symptoms):
    # Vectorize the input symptoms
    input_vectorized = vectorizer.transform([symptoms])

    predictions = {}
    for target in targets:
        predicted_encoded = models[target].predict(input_vectorized)
        predictions[target] = le_targets[target].inverse_transform(predicted_encoded)[0]

    return predictions

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf_document:
        for page in pdf_document:
            text += page.get_text()
    return text

# User input for PDF file
pdf_file_path = input("Please enter the path to the PDF file: ")

# Extract symptoms from the PDF
extracted_symptoms = extract_text_from_pdf(pdf_file_path)

# Predict outputs based on the extracted symptoms
predicted_outputs = predict_outputs(extracted_symptoms)

# Print the predicted outputs along with the disorder
print(f"Based on extracted symptoms from '{pdf_file_path}':")
print(f"Disorder: {predicted_outputs['Disorder']}")
print(f"Treatment Recommendation: {predicted_outputs['Treatment Recommendation']}")
print(f"Precautions: {predicted_outputs['Precautions']}")
print(f"Recommended Food Intake: {predicted_outputs['Food Intake Recommendation']}")


<ipython-input-4-603d82418174>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_data[target] = le_targets[target].fit_transform(target_data[target])
<ipython-input-4-603d82418174>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_data[target] = le_targets[target].fit_transform(target_data[target])
<ipython-input-4-603d82418174>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

Index(['Disorder', 'Symptoms', 'Treatment Recommendation',
       'Food Intake Recommendation', 'Precautions', 'Foods to Avoid',
       'Duration of Symptoms', 'Lifestyle Recommendations'],
      dtype='object')
Please enter the path to the PDF file: /content/PCOS_Medical_Report.pdf
Based on extracted symptoms from '/content/PCOS_Medical_Report.pdf':
Disorder: PCOS
Treatment Recommendation: Medications, surgery
Precautions: Maintain healthy weight, monitor cycles
Recommended Food Intake: Iron-rich foods


In [ ]:
!pip install pytesseract

In [ ]:
!pip install easyocr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 12.9 MB/s eta 0:00:00


In [ ]:
#uploading image
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import easyocr

# Load dataset
data = pd.read_csv('/content/gynecological_conditions.csv')

# View dataset columns to understand the structure
print(data.columns)

# Define input features and targets
X = data['Symptoms']  # Input features (Symptoms)

# Targets for prediction
targets = ['Disorder', 'Treatment Recommendation', 'Precautions', 'Food Intake Recommendation']
target_data = data[targets]

# Encoding targets
le_targets = {}
for target in targets:
    le_targets[target] = LabelEncoder()
    target_data[target] = le_targets[target].fit_transform(target_data[target])

# Split the dataset into training and testing sets for the features
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Get the corresponding target values for the training and testing sets
y_train = target_data.loc[X_train.index]
y_test = target_data.loc[X_test.index]

# Vectorize the symptoms data
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train separate Naive Bayes models for each target
models = {}
for target in targets:
    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train[target])  # Using y_train[target] to match the length of X_train_vectorized
    models[target] = model

# Save the trained models and vectorizer to files
with open('naive_bayes_treatment_model.pkl', 'wb') as model_file:
    pickle.dump(models, model_file)

with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

# Function to predict disorder, treatment, precautions, and food intake based on symptoms input
def predict_outputs(symptoms):
    # Vectorize the input symptoms
    input_vectorized = vectorizer.transform([symptoms])

    predictions = {}
    for target in targets:
        predicted_encoded = models[target].predict(input_vectorized)
        predictions[target] = le_targets[target].inverse_transform(predicted_encoded)[0]

    return predictions

# Function to extract text from an image using EasyOCR
def extract_text_from_image(image_path):
    reader = easyocr.Reader(['en'])  # Specify the language(s) you want to use
    result = reader.readtext(image_path)

    # Combine all detected text into a single string
    extracted_text = ' '.join([text[1] for text in result])
    return extracted_text

# User input for image file
image_file_path = input("Please enter the path to the image file: ")

# Extract symptoms from the image
extracted_symptoms = extract_text_from_image(image_file_path)

# Predict outputs based on the extracted symptoms
predicted_outputs = predict_outputs(extracted_symptoms)

# Print the predicted outputs along with the disorder
print(f"Based on extracted symptoms from '{image_file_path}':")
print(f"Disorder: {predicted_outputs['Disorder']}")
print(f"Treatment Recommendation: {predicted_outputs['Treatment Recommendation']}")
print(f"Precautions: {predicted_outputs['Precautions']}")
print(f"Recommended Food Intake: {predicted_outputs['Food Intake Recommendation']}")


<ipython-input-12-d4793360118e>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_data[target] = le_targets[target].fit_transform(target_data[target])
<ipython-input-12-d4793360118e>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_data[target] = le_targets[target].fit_transform(target_data[target])
<ipython-input-12-d4793360118e>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

Index(['Disorder', 'Symptoms', 'Treatment Recommendation',
       'Food Intake Recommendation', 'Precautions', 'Foods to Avoid',
       'Duration of Symptoms', 'Lifestyle Recommendations'],
      dtype='object')
Please enter the path to the image file: /content/DALL·E 2024-10-19 14.51.08 - A visual representation of symptoms including irregular periods, weight gain, and excess hair growth. The image depicts a female figure symbolizing th.jpg


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteBased on extracted symptoms from '/content/DALL·E 2024-10-19 14.51.08 - A visual representation of symptoms including irregular periods, weight gain, and excess hair growth. The image depicts a female figure symbolizing th.jpg':
Disorder: PCOS
Treatment Recommendation: Lifestyle changes, hormone treatments
Precautions: Maintain healthy weight, monitor cycles
Recommended Food Intake: Low-carb, high-fiber foods


In [ ]:
!pip install pandas scikit-learn tensorflow transformers


In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from tqdm import tqdm
from tensorflow.keras import layers, models  # Importing Keras components

# Load dataset
data = pd.read_csv('/content/gynecological_conditions.csv')  # Update with your dataset path

# Define input features and targets
X = data['Symptoms']  # Input features (Symptoms)

# Targets for prediction
targets = ['Disorder', 'Treatment Recommendation', 'Precautions', 'Food Intake Recommendation']
target_data = data[targets]

# Encoding targets
le_targets = {}
for target in targets:
    le_targets[target] = LabelEncoder()
    target_data[target] = le_targets[target].fit_transform(target_data[target])

# Split the dataset into training and testing sets for the features
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Get the corresponding target values for the training and testing sets
y_train = target_data.loc[X_train.index]
y_test = target_data.loc[X_test.index]

# Vectorize the symptoms data for Naive Bayes
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train and evaluate Naive Bayes Model
naive_bayes_model = MultinomialNB()
naive_bayes_model.fit(X_train_vectorized, y_train['Disorder'])  # Example for one target
y_pred_nb = naive_bayes_model.predict(X_test_vectorized)
nb_accuracy = accuracy_score(y_test['Disorder'], y_pred_nb)  # Example for one target
print(f"Naive Bayes Model Accuracy: {nb_accuracy:.4f}")

# Prepare tokenizer and model for LLM (BERT)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(le_targets['Disorder'].classes_))

# Function to create TensorFlow datasets
def create_tf_dataset(X, y):
    encodings = tokenizer(list(X), truncation=True, padding=True, max_length=128)
    dataset = tf.data.Dataset.from_tensor_slices((dict(encodings), y)).shuffle(1000).batch(16)
    return dataset

# Create TensorFlow datasets for LLM
train_dataset = create_tf_dataset(X_train, y_train['Disorder'])
test_dataset = create_tf_dataset(X_test, y_test['Disorder'])

# Set up optimizer and loss function using TensorFlow's Adam
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)  # Use TensorFlow's Adam optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Training Loop for BERT-based model
num_epochs = 3
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    for step, (batch_x, batch_y) in tqdm(enumerate(train_dataset)):
        with tf.GradientTape() as tape:
            logits = model(batch_x, training=True).logits
            loss = loss_fn(batch_y, logits)

        # Compute gradients and update model
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

# Evaluate the LLM model
y_pred_llm = []
for batch_x, _ in test_dataset:
    logits = model(batch_x, training=False).logits
    y_pred_llm.extend(np.argmax(logits, axis=1))

llm_accuracy = accuracy_score(y_test['Disorder'], y_pred_llm)
print(f"LLM Model (BERT) Accuracy: {llm_accuracy:.4f}")

# Deep Learning Model: Feedforward Neural Network
# Convert text data to TF-IDF features
X_train_dl = X_train_vectorized.toarray()  # Convert sparse matrix to dense
X_test_dl = X_test_vectorized.toarray()

# Build a Feedforward Neural Network
dl_model = models.Sequential([
    layers.Input(shape=(X_train_dl.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(le_targets['Disorder'].classes_), activation='softmax')  # Output layer
])

# Compile the model
dl_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
dl_model.fit(X_train_dl, y_train['Disorder'], epochs=3, batch_size=16, validation_split=0.1)

# Evaluate the deep learning model
y_pred_dl = np.argmax(dl_model.predict(X_test_dl), axis=1)
dl_accuracy = accuracy_score(y_test['Disorder'], y_pred_dl)
print(f"Deep Learning Model (Feedforward NN) Accuracy: {dl_accuracy:.4f}")

# Function to print accuracies for comparison
def print_accuracies():
    print("Model Accuracies:")
    print(f"Naive Bayes Accuracy: {nb_accuracy:.4f}")
    print(f"LLM (BERT) Accuracy: {llm_accuracy:.4f}")
    print(f"Deep Learning Model Accuracy: {dl_accuracy:.4f}")

# Call the function to display accuracies
print_accuracies()

# Save the label encoders for future predictions
for target in targets:
    with open(f'le_{target}.pkl', 'wb') as file:
        pickle.dump(le_targets[target], file)

# Save the vectorizer for future use
with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)


<ipython-input-18-e888b2ef582d>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_data[target] = le_targets[target].fit_transform(target_data[target])
<ipython-input-18-e888b2ef582d>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_data[target] = le_targets[target].fit_transform(target_data[target])
<ipython-input-18-e888b2ef582d>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

Naive Bayes Model Accuracy: 1.0000


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


360it [08:42,  1.45s/it]


Epoch 2/3


360it [08:30,  1.42s/it]


Epoch 3/3


360it [08:26,  1.41s/it]


LLM Model (BERT) Accuracy: 0.0458
Epoch 1/3
324/324 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8009 - loss: 1.7955 - val_accuracy: 1.0000 - val_loss: 0.0173
Epoch 2/3
324/324 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 1.0000 - loss: 0.0111 - val_accuracy: 1.0000 - val_loss: 0.0036
Epoch 3/3
324/324 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0029 - val_accuracy: 1.0000 - val_loss: 0.0016
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Deep Learning Model (Feedforward NN) Accuracy: 1.0000
Model Accuracies:
Naive Bayes Accuracy: 1.0000
LLM (BERT) Accuracy: 0.0458
Deep Learning Model Accuracy: 1.0000


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import fitz  # PyMuPDF for PDF text extraction

# Load dataset
data = pd.read_csv('/content/gynecological_conditions.csv')

# View dataset columns to understand the structure
print(data.columns)

# Define input features and targets
X = data['Symptoms']  # Input features (Symptoms)

# Targets for prediction (added more target columns)
targets = [
    'Disorder',
    'Treatment Recommendation',
    'Precautions',
    'Food Intake Recommendation',
    'Foods to Avoid',
    'Duration of Symptoms',
    'Lifestyle Recommendations'
]

# Ensure all target columns exist in the dataset
target_data = data[targets]

# Encoding targets
le_targets = {}
for target in targets:
    le_targets[target] = LabelEncoder()
    target_data[target] = le_targets[target].fit_transform(target_data[target])

# Split the dataset into training and testing sets for the features
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Get the corresponding target values for the training and testing sets
y_train = target_data.loc[X_train.index]
y_test = target_data.loc[X_test.index]

# Vectorize the symptoms data
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train separate Naive Bayes models for each target
models = {}
for target in targets:
    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train[target])  # Using y_train[target] to match the length of X_train_vectorized
    models[target] = model

# Save the trained models and vectorizer to files
with open('naive_bayes_treatment_model.pkl', 'wb') as model_file:
    pickle.dump(models, model_file)

with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

# Function to predict disorder, treatment, precautions, and food intake based on symptoms input
def predict_outputs(symptoms):
    # Vectorize the input symptoms
    input_vectorized = vectorizer.transform([symptoms])

    predictions = {}
    for target in targets:
        predicted_encoded = models[target].predict(input_vectorized)
        predictions[target] = le_targets[target].inverse_transform(predicted_encoded)[0]

    return predictions

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf_document:
        for page in pdf_document:
            text += page.get_text()
    return text

# User input for PDF file
pdf_file_path = input("Please enter the path to the PDF file: ")

# Extract symptoms from the PDF
extracted_symptoms = extract_text_from_pdf(pdf_file_path)

# Predict outputs based on the extracted symptoms
predicted_outputs = predict_outputs(extracted_symptoms)

# Print the predicted outputs along with the disorder in the specified format
print(f"Based on extracted symptoms from '{pdf_file_path}':")
print(f"Disorder: {predicted_outputs['Disorder']}")
print(f"Treatment Recommendation: {predicted_outputs['Treatment Recommendation']}")
print(f"Precautions: {predicted_outputs['Precautions']}")
print(f"Recommended Food Intake: {predicted_outputs['Food Intake Recommendation']}")
print(f"Foods to Avoid: {predicted_outputs['Foods to Avoid']}")
print(f"Duration of Symptoms: {predicted_outputs['Duration of Symptoms']}")
print(f"Lifestyle Recommendations: {predicted_outputs['Lifestyle Recommendations']}")


<ipython-input-4-84b1820d9729>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_data[target] = le_targets[target].fit_transform(target_data[target])
<ipython-input-4-84b1820d9729>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_data[target] = le_targets[target].fit_transform(target_data[target])
<ipython-input-4-84b1820d9729>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

Index(['Disorder', 'Symptoms', 'Treatment Recommendation',
       'Food Intake Recommendation', 'Precautions', 'Foods to Avoid',
       'Duration of Symptoms', 'Lifestyle Recommendations'],
      dtype='object')
Please enter the path to the PDF file: /content/PCOS_Medical_Report.pdf
Based on extracted symptoms from '/content/PCOS_Medical_Report.pdf':
Disorder: PCOS
Treatment Recommendation: Medications, surgery
Precautions: Maintain healthy weight, monitor cycles
Recommended Food Intake: Iron-rich foods
Foods to Avoid: Processed sugars
Duration of Symptoms: Varies
Lifestyle Recommendations: Regular exercise
